In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np

import pandas as pd
import os

In [2]:
train_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_df = pd.read_csv(train_path)
train_df

,cpsc_case_number,narrative,primary_mechanism
0,100145411,67 yof pt fell on the floor playing pickle bal...,"Falls, trips, etc."
1,100417928,63yof playing pickleball in the hot sun and pa...,Heat stroke
2,100417973,66yom strained lower leg playing pickle ball,Other mechanism
3,100417997,60yof pt playing pickle ball and fell sustaine...,"Falls, trips, etc."
4,100454409,70 yom pt injured knee while playing pickle ba...,Other mechanism
...,...,...,...
1986,200101825,68 yof tripped and fell while playing tennis. ...,"Falls, trips, etc."
1987,200102934,76yof was hit in the eye with a tennis ball dx...,Hit with various obj
1988,200110467,61yof was playing tennis when she developed pa...,Undetermined/unknown
1989,200124689,70yof fell while playing tennis dx: strained u...,"Falls, trips, etc."


In [3]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 'Multi-sport', 'Other')
print(train_labels_np[0:30])
print(train_labels_np_bin[0:30])

['Falls, trips, etc.' 'Heat stroke' 'Other mechanism' 'Falls, trips, etc.'
 'Other mechanism' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Hit with various obj' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'cardio/sync' 'Falls, trips, etc.' 'Other mechanism' 'Other mechanism'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Hit with various obj' 'Other mechanism'
 'Multi-sport' 'Falls, trips, etc.' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Falls, trips, etc.' 'Falls, trips, etc.']
['Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Multi-sport' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other']


In [4]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [5]:
maxlen = 207
training_samples = 1491
validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1857 unique tokens.
Shape of data tensor: (1991, 207)
Shape of label tensor: (1991,)


In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
train_labels_np = train_labels_np[indices]
x_train = data[:training_samples]
y_train = train_labels_np[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = train_labels_np[training_samples: training_samples + validation_samples]

In [8]:
label_encoder = LabelEncoder()

y_train_integer_encoded = label_encoder.fit_transform(y_train)
y_val_integer_encoded = label_encoder.fit_transform(y_val)

#y_train_integer_encoded
#y_train_integer_encoded
y_train_one_hot = to_categorical(y_train_integer_encoded, num_classes=7)
y_val_one_hot = to_categorical(y_val_integer_encoded, num_classes=7)

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(7, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 207, 8)            800000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1656)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 11599     
Total params: 811,599
Trainable params: 811,599
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(x_train, y_train_one_hot,
epochs=80,
batch_size=32,
validation_data=(x_val, y_val_one_hot))

Epoch 1/80
47/47 [==============================] - 1s 13ms/step - loss: 0.4024 - accuracy: 0.4668 - val_loss: 0.3197 - val_accuracy: 0.5060
Epoch 2/80
47/47 [==============================] - 0s 9ms/step - loss: 0.3256 - accuracy: 0.4769 - val_loss: 0.3069 - val_accuracy: 0.5060
Epoch 3/80
47/47 [==============================] - 0s 9ms/step - loss: 0.3111 - accuracy: 0.4863 - val_loss: 0.2924 - val_accuracy: 0.5400
Epoch 4/80
47/47 [==============================] - 0s 9ms/step - loss: 0.2910 - accuracy: 0.5218 - val_loss: 0.2711 - val_accuracy: 0.5620
Epoch 5/80
47/47 [==============================] - 0s 9ms/step - loss: 0.2678 - accuracy: 0.5714 - val_loss: 0.2502 - val_accuracy: 0.5840
Epoch 6/80
47/47 [==============================] - 0s 9ms/step - loss: 0.2461 - accuracy: 0.6278 - val_loss: 0.2338 - val_accuracy: 0.6140
Epoch 7/80
47/47 [==============================] - 0s 9ms/step - loss: 0.2276 - accuracy: 0.6720 - val_loss: 0.2166 - val_accuracy: 0.7240
Epoch 8/80
47/47 [=

In [14]:
modelv2 = Sequential()
modelv2.add(Embedding(max_words, 8, input_length=maxlen))
modelv2.add(Flatten())
modelv2.add(Dense(64, activation='relu'))
modelv2.add(Dense(7, activation='sigmoid'))
modelv2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
modelv2.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 207, 8)            800000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1656)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                106048    
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 455       
Total params: 906,503
Trainable params: 906,503
Non-trainable params: 0
_________________________________________________________________


In [15]:
history2= modelv2.fit(x_train, y_train_one_hot,
epochs=80,
batch_size=32,
validation_data=(x_val, y_val_one_hot))

Epoch 1/80
47/47 [==============================] - 1s 17ms/step - loss: 0.3662 - accuracy: 0.4688 - val_loss: 0.3125 - val_accuracy: 0.5060
Epoch 2/80
47/47 [==============================] - 0s 9ms/step - loss: 0.3163 - accuracy: 0.4816 - val_loss: 0.2921 - val_accuracy: 0.5180
Epoch 3/80
47/47 [==============================] - 0s 10ms/step - loss: 0.2859 - accuracy: 0.5292 - val_loss: 0.2566 - val_accuracy: 0.6280
Epoch 4/80
47/47 [==============================] - 0s 10ms/step - loss: 0.2513 - accuracy: 0.6284 - val_loss: 0.2296 - val_accuracy: 0.6540
Epoch 5/80
47/47 [==============================] - 0s 10ms/step - loss: 0.2242 - accuracy: 0.6821 - val_loss: 0.2076 - val_accuracy: 0.7320
Epoch 6/80
47/47 [==============================] - 0s 10ms/step - loss: 0.2042 - accuracy: 0.7210 - val_loss: 0.1930 - val_accuracy: 0.7540
Epoch 7/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1869 - accuracy: 0.7505 - val_loss: 0.1826 - val_accuracy: 0.7880
Epoch 8/80
47/4

In [ ]:
model.save_weights('../Model Objects/keras_outv2.h5')

In [ ]:
def getModel():
    my_dim = 8
    
    model = Sequential()
    model.add(Embedding(max_words, my_dim, input_length=maxlen))
    model.add(Flatten())
#    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [ ]:
# Different configurations to try
epochs_list = [30, 60, 90]
batch_sizes = [16, 32, 64]
#embedding_dim = 8
# Dictionary to store the history of each configuration
history_dict = {}

for epochs in epochs_list:
    for batch_size in batch_sizes:
        # Get a fresh instance of the model for each run
        model = getModel()

        # Compile the model
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        history = model.fit(x_train, y_train_one_hot,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, y_val_one_hot),
                            verbose=2)  # Set verbose to 2 for less output

        # Save the history of this configuration
        history_dict[f"epochs_{epochs}_batch_{batch_size}"] = history.history

        # Optionally, you can print out the final validation accuracy for each configuration
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Epochs: {epochs}, Batch size: {batch_size}, Validation accuracy: {final_val_accuracy}")

# Now you can analyze the history_dict to see which configuration performed best


In [ ]:
history_dict